# **Import and Install Neccessary Libraries**

In [ ]:
!pip install psycopg2
!pip install airflow

In [ ]:
import luigi
import pandas as pd
import psycopg2

# **Luigi ETL**

In [ ]:

class CombineFiles(luigi.Task):
    def requires(self):
        return None

    def output(self):
        return luigi.LocalTarget('Customer_LTV_Data_Composite.csv')

    def run(self):
        df1 = pd.read_csv('Customer_LTV_Data_1.csv')
        df2 = pd.read_csv('Customer_LTV_Data_2.csv')
        df = pd.concat([df1, df2])
        df.to_csv('Customer_LTV_Data_Composite.csv', index=False)

class ExportToPostgres(luigi.Task):
    host = luigi.Parameter()
    database = luigi.Parameter()
    user = luigi.Parameter()
    password = luigi.Parameter()

    def requires(self):
        return CombineFiles()

    def run(self):
        connection = psycopg2.connect(
            host=self.host,
            database=self.database,
            user=self.user,
            password=self.password
        )

        cursor = connection.cursor()

        with self.input().open('r') as f:
            cursor.copy_from(f, 'customer_ltv_data', sep=',')

        connection.commit()

    def output(self):
        return luigi.LocalTarget('customer_ltv_data_loaded.txt')

if __name__ == '__main__':
    host = input("Enter Postgres host: ")
    database = input("Enter Postgres database: ")
    user = input("Enter Postgres user: ")
    password = input("Enter Postgres password: ")

    luigi.build([ExportToPostgres(host=host, database=database, user=user, password=password)], force=True)